In [1]:
import pandas as pd
import pickle
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta


from tqdm.notebook import tqdm
tqdm.pandas()
from utils import full_clean, create_subframe, create_t3_df, create_t1_df, get_parallel_edges, create_complete_df, merge_df,\
    get_parallel_edges, get_weighted_edges, get_edge_info, create_post_comment_name_id

/Users/victor/opt/anaconda3/envs/ust/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# Import the bot list retrieved online

In [2]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

# Import the top spammers who are considered as bots

In [3]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/10k_sub_final.csv')
popular_bot = popular_author[popular_author.is_bot == 1]

--------
<h1>Proper work</h1>

In [ ]:
# #month walker
# def month_walker(comment_df, post_df, year, month):
#     month_info = dict()
#     concat_df = merge_df(comment_df, post_df)
#     #clean dataframe
#     bot, cleaned_df = full_clean(concat_df, bot_list, popular_bot, by_parent=True)
#     #get the output df list
#     df_list = create_subframe(year, month, cleaned_df)
#     day = 1
#     for daydf in df_list:
#         #get t3_df
#         t3_name_id, t3_df = create_t3_df(daydf)
#         #get t1_df
#         t1_name_id, t1_df = create_t1_df(daydf)
#         #concat the t1 and t3 df to get the complete dataframe
#         complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)
        
#         #get weighted edge list   
#         parallel_edges = get_parallel_edges(complete_df, daydf)
#         weighted_edges = get_weighted_edges(parallel_edges)
#         weighted_edges['inverse_weight'] = 1/weighted_edges['weight']
        
#         # get the information
#         # get the list of post author names
#         daydf = daydf[daydf['parent_id']==0]
#         post_authors = daydf['author'].to_numpy()
#         edge_info = get_edge_info(weighted_edges, post_authors)
        
        
#         #this saves the df to the month network list
#         month_info.update({day: edge_info})
#         day += 1
        
#     return month_info

In [ ]:
def month_walker(comment_df, post_df, year, month):
    # clean the comment df
    comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot, by_parent=True)
    # clean the post df
    post_bot, post_df = full_clean(post_df, bot_list, popular_bot)
    # cut the post df by month
    startdate = datetime(year, month, 1, 0)
    enddate = startdate + relativedelta(month=1)
    post_df = post_df[(post_df.created_utc >= startdate.timestamp()) & (post_df.created_utc < enddate.timestamp())]
    # create the t1 and t3 id:name key value pair
    t3_name_id, t1_name_id = create_post_comment_name_id(post_df, comment_df)
    # concat the comment and post df
    concat_df = merge_df(comment_df, post_df)
    # create the t1 and t3 df
    t1_df = create_t1_df(concat_df)
    t3_df = create_t3_df(concat_df)
    # create the complete df
    complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)
    # create the parallel and weighted edgelist
    parallel_edges = get_parallel_edges(complete_df, concat_df)
    weighted_edges = get_weighted_edges(parallel_edges)
    weighted_edges['inverse_weight'] = 1/weighted_edges['weight']
    
    # save the edgelists ......

----------
<h1>Test</h1>

In [4]:
fp_comment = '/Users/victor/Desktop/CS/UST_summer/01-20_comments.csv'
comment_df = pd.read_csv(fp_comment, low_memory = False)[['author', 'id', 'created_utc', 'link_id', 'parent_id']]
fp_post = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv'
post_df = pd.read_csv(fp_post, low_memory = False)[['author', 'id', 'created_utc']]
# cut the post_df into month
post_df = post_df[(post_df.created_utc >= 1609459200) & (post_df.created_utc < 1612137600)]

In [5]:
comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot, by_parent=True)

  0%|          | 0/452604 [00:00<?, ?it/s]

In [6]:
post_bot, post_df = full_clean(post_df, bot_list, popular_bot)

  0%|          | 0/373878 [00:00<?, ?it/s]

In [7]:
t3_name_id, t1_name_id = create_post_comment_name_id(post_df, comment_df)

In [8]:
concat_df = merge_df(comment_df, post_df)

In [9]:
t3_df = create_t3_df(concat_df)

/Users/victor/Desktop/CS/UST_summer/utils/dataframe_utils.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level_1_comments['merged'] = level_1_comments.apply(lambda x: {x['author']:x['created_utc']}, axis=1) #merge author and created_utc into dict


  0%|          | 0/5181 [00:00<?, ?it/s]

In [10]:
t1_df = create_t1_df(concat_df)

/Users/victor/Desktop/CS/UST_summer/utils/dataframe_utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_comments['merged'] = sub_comments.apply(lambda x: {x['author']:x['created_utc']}, axis=1) #merge author and created_utc into dict


  0%|          | 0/33625 [00:00<?, ?it/s]

In [11]:
complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)

In [ ]:
author_list = complete_df.merged.iloc[0]
author_list

In [ ]:
author = author_list[0]
k,v = next((str(k), str(v)) for k, v in author.items())
type(k)
    

In [12]:
parallel_edges = get_parallel_edges(complete_df, concat_df)
weighted_edges = get_weighted_edges(parallel_edges)
weighted_edges['inverse_weight'] = 1/weighted_edges['weight']